# Store Sales - Time Series Forcasting
### Samuel Zamudio & Mendez

In [1]:
# import common libraries
import pandas as pd
import numpy as np

In [2]:
# import data
holidays_events_df = pd.read_csv('./holidays_events.csv')
oil_df = pd.read_csv('./oil.csv')
stores_df = pd.read_csv('./stores.csv')
test_df = pd.read_csv('./test.csv')
train_df = pd.read_csv('./train.csv')
transactions_df = pd.read_csv('./transactions.csv')


In [3]:
train_df.duplicated().sum()

0

## Exploratory Data Analysis 

## Holiday Events

In [4]:
holidays_events_df.head(10)

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
5,2012-05-12,Holiday,Local,Puyo,Cantonizacion del Puyo,False
6,2012-06-23,Holiday,Local,Guaranda,Cantonizacion de Guaranda,False
7,2012-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura,False
8,2012-06-25,Holiday,Local,Latacunga,Cantonizacion de Latacunga,False
9,2012-06-25,Holiday,Local,Machala,Fundacion de Machala,False


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           28512 non-null  int64 
 1   date         28512 non-null  object
 2   store_nbr    28512 non-null  int64 
 3   family       28512 non-null  object
 4   onpromotion  28512 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.1+ MB


In [7]:
oil_df.head(10)

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
5,2013-01-08,93.21
6,2013-01-09,93.08
7,2013-01-10,93.81
8,2013-01-11,93.60
9,2013-01-14,94.27


## Data Preprocessing

In [ ]:
holidays_events_df['description'].unique()

## Concatenate Train and Test Data

In [8]:
# Adding identifier columns
train_df['data_id'] = 'df1'
test_df['data_id'] = 'df2'

# Concatenating the DataFrames
combined_df = pd.concat([train_df, test_df], ignore_index=True)

In [9]:
combined_df.head(10)

,id,date,store_nbr,family,sales,onpromotion,data_id
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,df1
1,1,2013-01-01,1,BABY CARE,0.0,0,df1
2,2,2013-01-01,1,BEAUTY,0.0,0,df1
3,3,2013-01-01,1,BEVERAGES,0.0,0,df1
4,4,2013-01-01,1,BOOKS,0.0,0,df1
5,5,2013-01-01,1,BREAD/BAKERY,0.0,0,df1
6,6,2013-01-01,1,CELEBRATION,0.0,0,df1
7,7,2013-01-01,1,CLEANING,0.0,0,df1
8,8,2013-01-01,1,DAIRY,0.0,0,df1
9,9,2013-01-01,1,DELI,0.0,0,df1


In [10]:
combined_df.isna().sum()

id                 0
date               0
store_nbr          0
family             0
sales          28512
onpromotion        0
data_id            0
dtype: int64

In [11]:
# Casting to datetime
combined_df['date'] = pd.to_datetime(combined_df['date'])
print(combined_df.dtypes)

id                      int64
date           datetime64[ns]
store_nbr               int64
family                 object
sales                 float64
onpromotion             int64
data_id                object
dtype: object


In [12]:
# Make sure oil_df date is datetime type
oil_df['date'] = pd.to_datetime(oil_df['date'])
print(len(combined_df))

# Merging DataFrames on 'Date'
combined_df = combined_df.merge(oil_df, on='date', how='left')
combined_df.sample(10)
print(len(combined_df))

3029400
3029400


combined_df

In [13]:
# Merge with store data
print(len(combined_df))
combined_df = combined_df.merge(stores_df, on='store_nbr', how='left')
combined_df.sample(10)
print(len(combined_df))

3029400
3029400


In [14]:
transactions_df['date'] = pd.to_datetime(transactions_df['date'])
print(len(combined_df))

# Merge with trasaction data
combined_df = combined_df.merge(transactions_df, on=['date', 'store_nbr'], how='left')
combined_df.sample(10)

print(len(combined_df))

3029400
3029400


In [15]:
combined_df

,id,date,store_nbr,family,sales,onpromotion,data_id,dcoilwtico,city,state,type,cluster,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,df1,NaN,Quito,Pichincha,D,13,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,df1,NaN,Quito,Pichincha,D,13,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,df1,NaN,Quito,Pichincha,D,13,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,df1,NaN,Quito,Pichincha,D,13,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,df1,NaN,Quito,Pichincha,D,13,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,NaN,1,df2,47.26,Quito,Pichincha,B,6,NaN
3029396,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,df2,47.26,Quito,Pichincha,B,6,NaN
3029397,3029397,2017-08-31,9,PRODUCE,NaN,1,df2,47.26,Quito,Pichincha,B,6,NaN
3029398,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,df2,47.26,Quito,Pichincha,B,6,NaN


In [16]:
# Create new columns for month, day, and year
combined_df['month'] = combined_df['date'].dt.month
combined_df['day'] = combined_df['date'].dt.day
combined_df['year'] = combined_df['date'].dt.year

In [18]:
combined_df.to_csv('./combined_df.csv', index = True)

In [19]:
print(len(combined_df))

holidays_events_df['date'] = pd.to_datetime(holidays_events_df['date'])

# Create new columns for month, day, and year
holidays_events_df['month'] = holidays_events_df['date'].dt.month
holidays_events_df['day'] = holidays_events_df['date'].dt.day

# Filter out transferred holidays (where transferred is True)
holidays_events_df = holidays_events_df[(holidays_events_df['transferred'].isna()) | (holidays_events_df['transferred'] == False)]

# Separate DataFrame for national holidays
national_holidays = holidays_events_df[holidays_events_df['type'] == 'National']

# DataFrame for local holidays
local_holidays = holidays_events_df[holidays_events_df['type'] != 'National']

# Aggregating national_holidays
national_holidays = national_holidays.groupby(['day', 'month']).agg('first').reset_index()

# Aggregating local_holidays
local_holidays = local_holidays.groupby(['day', 'month', 'locale_name']).agg('first').reset_index()

# Merge for national holidays
combined_df = combined_df.merge(national_holidays[['day', 'month']], on=['day', 'month'], how='left', indicator=True)
combined_df['was_holiday'] = combined_df['_merge'] == 'both'
combined_df.drop('_merge', axis=1, inplace=True)

# Merge for local holidays
combined_df = combined_df.merge(local_holidays[['day', 'month', 'locale_name']], left_on=['day', 'month', 'city'], right_on=['day', 'month', 'locale_name'], how='left', indicator=True)
combined_df['was_holiday'] = combined_df['was_holiday'] | (combined_df['_merge'] == 'both')
combined_df.drop(['_merge', 'locale_name'], axis=1, inplace=True)

display(combined_df[combined_df['was_holiday'] == True])
display(holidays_events_df)

print(len(combined_df))


3029400


,id,date,store_nbr,family,sales,onpromotion,data_id,dcoilwtico,city,state,type,cluster,transactions,month,day,year,was_holiday
108471,108471,2013-03-02,52,AUTOMOTIVE,0.0,0,df1,NaN,Manta,Manabi,A,11,NaN,3,2,2013,True
108472,108472,2013-03-02,52,BABY CARE,0.0,0,df1,NaN,Manta,Manabi,A,11,NaN,3,2,2013,True
108473,108473,2013-03-02,52,BEAUTY,0.0,0,df1,NaN,Manta,Manabi,A,11,NaN,3,2,2013,True
108474,108474,2013-03-02,52,BEVERAGES,0.0,0,df1,NaN,Manta,Manabi,A,11,NaN,3,2,2013,True
108475,108475,2013-03-02,52,BOOKS,0.0,0,df1,NaN,Manta,Manabi,A,11,NaN,3,2,2013,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3016657,3016657,2017-08-24,50,POULTRY,NaN,0,df2,47.24,Ambato,Tungurahua,A,14,NaN,8,24,2017,True
3016658,3016658,2017-08-24,50,PREPARED FOODS,NaN,0,df2,47.24,Ambato,Tungurahua,A,14,NaN,8,24,2017,True
3016659,3016659,2017-08-24,50,PRODUCE,NaN,2,df2,47.24,Ambato,Tungurahua,A,14,NaN,8,24,2017,True
3016660,3016660,2017-08-24,50,SCHOOL AND OFFICE SUPPLIES,NaN,13,df2,47.24,Ambato,Tungurahua,A,14,NaN,8,24,2017,True


,date,type,locale,locale_name,description,transferred,month,day
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,3,2
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,4,1
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,4,12
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False,4,14
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,4,21
...,...,...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False,12,22
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False,12,23
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False,12,24
348,2017-12-25,Holiday,National,Ecuador,Navidad,False,12,25


3029400


In [20]:
# impute the missing values for the oil prices using KNN
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

impute_df = combined_df.copy()

# select columns
columns = ['store_nbr', 'family',
       'onpromotion', 'dcoilwtico', 'city', 'state', 'type',
       'cluster', 'transactions', 'month', 'day', 'year']

impute_df = impute_df[columns]


# choose columns to encode
categorical_cols = ['family','city','state','type']
numeric_cols = [col for col in impute_df.columns if col not in categorical_cols]

# preprocess df
preprocessor = ColumnTransformer(
    transformers = [
        ('num', 'passthrough', numeric_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ]
)

encoded_combined = preprocessor.fit_transform(impute_df)

# impute missing values using KNN
imputer = KNNImputer(n_neighbors=3)

imputed_combined = imputer.fit_transform(encoded_combined.toarray())

# Get the column names after one-hot encoding
encoded_columns = preprocessor.named_transformers_['cat'].get_feature_names_out(input_features=categorical_cols)
all_columns = np.concatenate([numeric_cols, encoded_columns])

# Create DataFrame using the correct column names
imputed_combined_df = pd.DataFrame(imputed_combined, columns=all_columns)


In [ ]:
impurted_combined_df.to_csv('./imputed_combined_df.csv', index = True)

In [ ]:
combined_df['dcoilwtico'] = imputed_combined_df['dcoilwtico']

In [ ]:
indexes_are_aligned = combined_df.index.equals(imputed_combined_df.index)
print(indexes_are_aligned)